In [2]:
# !pip install openpyxl

# Imports

In [3]:
import pandas as pd
import openpyxl 

# Data

In [4]:
dir_path = "C:/Users/CPL17/OneDrive/Desktop/"
full_path = dir_path + "PA Wildflower Database.xlsx"

In [5]:
article_long = pd.read_excel(full_path,sheet_name="ARTICLE")


In [6]:
metadata= pd.read_excel(full_path,sheet_name="ERA_PA")

In [7]:
article_long.shape

(698, 11)

In [8]:
article_long.head()

,USDA Symbol,Source,Source URL,Common Plant,Scientific,QA,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,MODI,Audobon Birdy Dozen 2022,https://pa.audubon.org/sites/default/files/bir...,Scarlet Beebalm,Monarda didyma,1.0,NaN,NaN,NaN,NaN,NaN
1,LISP,Audobon Birdy Dozen 2022,https://pa.audubon.org/sites/default/files/bir...,Dense Blazing Star,Liatris spicata,1.0,NaN,NaN,NaN,NaN,NaN
2,LOSI,Audobon Birdy Dozen 2022,https://pa.audubon.org/sites/default/files/bir...,Great Blue Lobelia,Lobelia siphilitica,1.0,NaN,NaN,NaN,NaN,NaN
3,IRCR,Audobon Birdy Dozen 2022,https://pa.audubon.org/sites/default/files/bir...,Dwarf Crested Iris,Iris cristata,1.0,NaN,NaN,NaN,NaN,NaN
4,CHVI5,Audobon Birdy Dozen 2022,https://pa.audubon.org/sites/default/files/bir...,Green And Gold,Chrysogonum virginianum,1.0,NaN,NaN,NaN,NaN,NaN


In [9]:
article_long["Source"].value_counts()

Plants for Birds                                          107
Salt Tolerant Plants                                       93
Deer Resistant                                             75
PA Rare, Threatened, Endangered, and Extirpated Plants     52
Rain Gardens                                               51
Witchcraft Plants                                          48
Under Black Walnut Trees                                   42
Suggested Hedgrow Plants                                   39
Slopes and Hills                                           25
Native Plants for Alkaline sites                           24
Native Plants for Septic or Sand Mounds                    22
Tried & True                                               21
Edible Plants                                              19
Butterfly Basics                                           13
Audobon Birdy Dozen 2022                                   12
Groundcovers                                               10
Woodland

In [10]:
article_long["USDA Symbol"].value_counts().iloc[[0,-1]]

SYNO2    8
RHCA4    1
Name: USDA Symbol, dtype: int64

In [11]:
#Drop Missing

article_long.dropna(subset=["USDA Symbol"],inplace=True)

# Aggregations

In [12]:
f = lambda x: ', '.join(x)
article_agg = article_long.groupby("USDA Symbol").agg({"Source":list,"Source URL":list})

In [13]:
article_agg.head()

,Source,Source URL
USDA Symbol,,
ACAM,[Witchcraft Plants],[https://www.google.com/search?q=ACAM+Witchcra...
ACMI2,[Witchcraft Plants],[https://www.google.com/search?q=ACMI2+Witchcr...
ACNE2,[Salt Tolerant Plants],[https://edgeofthewoodsnursery.com/wp-content/...
ACPE,[Plants for Birds],[https://edgeofthewoodsnursery.com/wp-content/...
ACRA7,"[Tried & True, Woodland Mix (tall)]",[https://bhwp.org/grow/garden-with-natives/get...


In [14]:
article_agg.reset_index(inplace=True)
article_agg.columns = ["USDA Symbol","Source","Source URL"]
article_agg["Count"] = article_agg.Source.apply(lambda x: len(x))

# f = lambda x: ', '.join(map(str, set(x)))

for column in ["Source","Source URL"]:
    article_agg[column] = article_agg[column].apply(lambda x: ", ".join(x))

article_agg.head()

,USDA Symbol,Source,Source URL,Count
0,ACAM,Witchcraft Plants,https://www.google.com/search?q=ACAM+Witchcraf...,1
1,ACMI2,Witchcraft Plants,https://www.google.com/search?q=ACMI2+Witchcra...,1
2,ACNE2,Salt Tolerant Plants,https://edgeofthewoodsnursery.com/wp-content/u...,1
3,ACPE,Plants for Birds,https://edgeofthewoodsnursery.com/wp-content/u...,1
4,ACRA7,"Tried & True, Woodland Mix (tall)",https://bhwp.org/grow/garden-with-natives/get-...,2


In [15]:
(article_agg["USDA Symbol"].value_counts() != 1).sum() #Ensure Properly Aggregate

0

In [16]:
article_agg.sort_values(by="Count").iloc[[0,-1]]

,USDA Symbol,Source,Source URL,Count
0,ACAM,Witchcraft Plants,https://www.google.com/search?q=ACAM+Witchcraf...,1
306,SYNO2,"Under Black Walnut Trees, Tried & True, Butter...",http://www.ecosystemgardening.com/what-to-plan...,8


# Join with metdata

In [17]:
metadata = metadata[["USDA Symbol","Scientific Name","Common Name"]]
metadata.columns = ["USDA Symbol","Scientific Name","Common Name"]

In [18]:
metadata.head(3)

,USDA Symbol,Scientific Name,Common Name
0,ABBA,Abies balsamea,Balsam Fir
1,ACNE2,Acer negundo,Boxelder
2,ACPE,Acer pensylvanicum,Striped Maple


In [19]:
final = article_agg.merge(metadata,on="USDA Symbol",how="left")
final.head()

,USDA Symbol,Source,Source URL,Count,Scientific Name,Common Name
0,ACAM,Witchcraft Plants,https://www.google.com/search?q=ACAM+Witchcraf...,1,Acorus americanus,Sweetflag
1,ACMI2,Witchcraft Plants,https://www.google.com/search?q=ACMI2+Witchcra...,1,Achillea millefolium,Common Yarrow
2,ACNE2,Salt Tolerant Plants,https://edgeofthewoodsnursery.com/wp-content/u...,1,Acer negundo,Boxelder
3,ACPE,Plants for Birds,https://edgeofthewoodsnursery.com/wp-content/u...,1,Acer pensylvanicum,Striped Maple
4,ACRA7,"Tried & True, Woodland Mix (tall)",https://bhwp.org/grow/garden-with-natives/get-...,2,Actaea racemosa,Black Baneberry


In [20]:
missing_symbols = set(article_agg["USDA Symbol"]).difference(set(metadata["USDA Symbol"]))
missing_symbols

{'CAPL5', 'MECO', 'SPMA3', 'STDI3', 'ceca4', 'frvi', 'ruoc', 'viopa2'}

The symbol listed above are 8 symbols not present in the Source Data

In [21]:
cond = final["USDA Symbol"].isin(missing_symbols)
final[cond].head()

,USDA Symbol,Source,Source URL,Count,Scientific Name,Common Name
57,CAPL5,Small Grasses - Meadow Like,https://bhwp.org/grow/garden-with-natives/get-...,1,NaN,NaN
159,MECO,Woodland Mix (short),https://bhwp.org/grow/garden-with-natives/get-...,1,NaN,NaN
291,SPMA3,Woodland Mix (tall),https://bhwp.org/grow/garden-with-natives/get-...,1,NaN,NaN
293,STDI3,Woodland Mix (tall),https://bhwp.org/grow/garden-with-natives/get-...,1,NaN,NaN
346,ceca4,Under Black Walnut Trees,http://www.ecosystemgardening.com/what-to-plan...,1,NaN,NaN


In [22]:
final = final[~cond]

In [23]:
final["Common Name"] = final["Common Name"].str.title()

In [30]:
final["String"] = [f"{row['Common Name']} ({row['Scientific Name']}): {row['Source']}" for _,row in final.iterrows()]

In [ ]:
# final.loc[cond,"Scientific Name"] = ["Celtis tenuifolia","Gaura biennis L.","Gaura longiflora"]

In [ ]:
# final["String"] = [f"{row['Common Name']} ({row['Scientific Name']}): {row['WEB']}" for _,row in final.iterrows()]


In [31]:
final.head()

,USDA Symbol,Source,Source URL,Count,Scientific Name,Common Name,String
0,ACAM,Witchcraft Plants,https://www.google.com/search?q=ACAM+Witchcraf...,1,Acorus americanus,Sweetflag,Sweetflag (Acorus americanus): Witchcraft Plants
1,ACMI2,Witchcraft Plants,https://www.google.com/search?q=ACMI2+Witchcra...,1,Achillea millefolium,Common Yarrow,Common Yarrow (Achillea millefolium): Witchcra...
2,ACNE2,Salt Tolerant Plants,https://edgeofthewoodsnursery.com/wp-content/u...,1,Acer negundo,Boxelder,Boxelder (Acer negundo): Salt Tolerant Plants
3,ACPE,Plants for Birds,https://edgeofthewoodsnursery.com/wp-content/u...,1,Acer pensylvanicum,Striped Maple,Striped Maple (Acer pensylvanicum): Plants for...
4,ACRA7,"Tried & True, Woodland Mix (tall)",https://bhwp.org/grow/garden-with-natives/get-...,2,Actaea racemosa,Black Baneberry,Black Baneberry (Actaea racemosa): Tried & Tru...


In [35]:
final.to_excel("ARTICLE_AGG.xlsx")